# H20 library with scaler
Initialize the connection with the H2O cluster, obtaining a summary of the cluster status.

In [1]:
import h2o
import pandas as pd

from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from src.f_clean import *

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /home/b/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpquxztl22
  JVM stdout: /tmp/tmpquxztl22/h2o_b_started_from_python.out
  JVM stderr: /tmp/tmpquxztl22/h2o_b_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.2
H2O_cluster_version_age:,12 days
H2O_cluster_name:,H2O_from_python_b_2sxdo1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.916 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


H2O has several supervised and unsupervised models, and two automodels:
* automl: Retrieves information about an AutoML instance.
* H2OAutoML: Automates the supervised machine learning model training process.--> We will use this one!
## 1. Data cleaning
We have two options to clean the categorical columns:
* Get dummy columns, so we create a new column for each category. We will get dummy columns for color and clarity.
* Categorizing the column, to give it a numerical value. The advantage here is that we don't add any new columns. We will use it for the cut.

In [3]:
diamonds = pd.read_csv("../data/diamonds_train.csv")

# Drop column
diamonds.drop(columns = "Unnamed: 0", inplace = True)

# Categorizing the cut
cut_categ = ('Fair', "Good", 'Very Good', 'Premium', 'Ideal')
diamonds.cut = pd.Categorical(diamonds.cut, ordered = True, categories = cut_categ)
diamonds['numeric_cut'] = diamonds.cut.cat.codes
diamonds.drop(columns = "cut", inplace = True)

# Eliminating outliers
diamonds = diamonds[(diamonds["y"] != 0) & (diamonds["z"] != 0) & (diamonds["y"] < 30)]

# Get dummies
diamonds = pd.get_dummies(diamonds)

diamonds.reset_index(inplace=True)
diamonds.drop(columns = "index", inplace=True)
diamonds.head()

,carat,depth,table,x,y,z,price,numeric_cut,color_D,color_E,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,1.21,63.0,57.0,6.73,6.70,4.23,6134,4,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0.28,64.0,56.0,4.14,4.17,2.66,532,2,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0.42,61.2,58.0,4.86,4.82,2.96,1103,3,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0.26,61.1,57.0,4.16,4.12,2.53,600,4,0,0,...,0,0,0,1,0,0,0,0,0,0
4,1.10,63.4,57.0,6.52,6.55,4.14,4997,1,0,0,...,0,0,0,0,1,0,0,0,0,0


Create the scaler fitting it to the diamonds df, and scale diamonds.

In [4]:
scaler = MinMaxScaler() #StandardScaler()
scaler.fit(diamonds.drop(columns = "price")) # price won't be scaled!
diamonds_scaled = scaler.transform(diamonds.drop(columns = "price")) #np.array

cols1 = list(diamonds.columns[0:6])
cols2 = list(diamonds.columns[7:])
diamonds_scaled = pd.DataFrame(diamonds_scaled, columns = cols1 + cols2) 
diamonds_scaled["price"] = diamonds["price"]
diamonds_scaled.head()

,carat,depth,table,x,y,z,numeric_cut,color_D,color_E,color_F,...,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2,price
0,0.265092,0.555556,0.269231,0.476948,0.479936,0.595104,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,6134
1,0.020997,0.583333,0.250000,0.065183,0.073836,0.299435,0.50,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,532
2,0.057743,0.505556,0.288462,0.179650,0.178170,0.355932,0.75,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1103
3,0.015748,0.502778,0.269231,0.068362,0.065811,0.274953,1.00,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,600
4,0.236220,0.566667,0.269231,0.443561,0.455859,0.578154,0.25,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4997


## 2. Data training
In sklearn the set is divided into X (features) and y (groundtruth), and each of them into train and test.
In H2O the set is divided between train and test, and the name of the columns of X and y is passed.

In [5]:
colsX = list(diamonds_scaled.drop(columns = "price").columns)
colsy = "price"

train = diamonds_scaled[0:33000] 
test = diamonds_scaled[33000:]

'training_frame' and test_data must be a valid H2OFrame, so we have two options:
* Import train and test as csv with h2o.importFile("C:/MyData/h2otrain.csv")
* Convert train and test into a H2OFrame, passing a python object

In [6]:
train_h20frame = h2o.H2OFrame(train)
test_h20frame = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


There are two stop parameters in H2OAutoML: the max_runtime_secs and max_models. We will use max_models only, as we are getting better results with it.

In [7]:
#aml = H2OAutoML(max_runtime_secs = 120)
aml = H2OAutoML(max_models=30, seed=1) # con 40 modelos tarda 2h 30

In [8]:
# Train the models
aml.train(x = colsX, y = colsy, training_frame = train_h20frame)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [9]:
# Get the leaderboard table
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200511_232027,287895,536.558,287895,272.072,0.0980497
StackedEnsemble_AllModels_AutoML_20200511_232027,288470,537.094,288470,274.32,0.0998325
GBM_1_AutoML_20200511_232027,308207,555.164,308207,288.113,0.105705
GBM_2_AutoML_20200511_232027,310978,557.654,310978,288.651,0.105407
GBM_4_AutoML_20200511_232027,311512,558.133,311512,280.203,0.0967226
GBM_3_AutoML_20200511_232027,314008,560.364,314008,285.405,0.101664
DRF_1_AutoML_20200511_232027,317362,563.349,317362,283.153,0.0976242
XRT_1_AutoML_20200511_232027,318014,563.927,318014,282.125,0.0976936
XGBoost_grid__1_AutoML_20200511_232027_model_6,319104,564.893,319104,283.783,0.0973387
GBM_5_AutoML_20200511_232027,326242,571.176,326242,289.645,nan


## 3. Predict "price" for the training set

In [10]:
y_pred = aml.leader.predict(test_h20frame)
y_pred_df = y_pred.as_data_frame()
y_pred_df.head()

stackedensemble prediction progress: |████████████████████████████████████| 100%


,predict
0,11322.389390
1,7991.027611
2,8665.901355
3,627.920540
4,12772.406252


In [11]:
y_test = test[["price"]]
y_test.head()

,price
33000,9540
33001,7695
33002,7103
33003,537
33004,12433


#### Calculate the rmse

In [12]:
from sklearn.metrics import mean_squared_error
import math

In [13]:
mse = mean_squared_error(y_test, y_pred_df)

rmse = math.sqrt(mse)
print(f"rmse = {round(rmse, 5)}") # Root Mean Squared Error

rmse = 518.26428


## 4. Validation of the model
Remove, categorize and get dummies for the same columns of the training set.

In [14]:
validation = pd.read_csv("../data/diamonds_test.csv")

# Drop column
validation.drop(columns = "Unnamed: 0", inplace = True)

# Categorizing the cut
cut_categ = ('Fair', "Good", 'Very Good', 'Premium', 'Ideal')
validation.cut = pd.Categorical(validation.cut, ordered = True, categories = cut_categ)
validation['numeric_cut'] = validation.cut.cat.codes
validation.drop(columns = "cut", inplace = True)

# Get dummies
validation = pd.get_dummies(validation)

validation.head()

,carat,depth,table,x,y,z,numeric_cut,color_D,color_E,color_F,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.30,60.0,56.0,4.41,4.43,2.65,4,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0.34,62.1,57.0,4.52,4.46,2.79,4,1,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1.57,60.3,58.0,7.58,7.55,4.56,2,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,0.31,61.8,57.0,4.32,4.36,2.68,4,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1.51,64.0,60.0,7.26,7.21,4.63,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0


Normalize the columns.

In [15]:
validation_scaled = scaler.transform(validation)
validation_scaled = pd.DataFrame(validation_scaled, columns = validation.columns)
validation_scaled

,carat,depth,table,x,y,z,numeric_cut,color_D,color_E,color_F,...,color_I,color_J,clarity_I1,clarity_IF,clarity_SI1,clarity_SI2,clarity_VS1,clarity_VS2,clarity_VVS1,clarity_VVS2
0,0.026247,0.472222,0.250000,0.108108,0.115570,0.297552,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.036745,0.530556,0.269231,0.125596,0.120385,0.323917,1.00,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.359580,0.480556,0.288462,0.612083,0.616372,0.657250,0.50,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.028871,0.522222,0.269231,0.093800,0.104334,0.303202,1.00,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,0.343832,0.583333,0.326923,0.561208,0.561798,0.670433,0.25,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13444,0.223097,0.480556,0.230769,0.448331,0.476726,0.549906,1.00,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
13445,0.018373,0.525000,0.250000,0.063593,0.072231,0.280603,0.50,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
13446,0.341207,0.522222,0.326923,0.567568,0.587480,0.651601,0.75,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
13447,0.393701,0.477778,0.346154,0.645469,0.637239,0.676083,0.75,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Convert the df validation_scaled to H2OFrame.

In [16]:
validation_test = h2o.H2OFrame(validation_scaled)

Parse progress: |█████████████████████████████████████████████████████████| 100%


Predict the price values for the validation set.

In [17]:
y_pred_val = aml.leader.predict(validation_test)
y_pred_val

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
402.403
2054.63
9595.7
555.586
9529.74
2258.44
1045.93
10057.5
932.804
528.68


In [18]:
# y_pred needs to be converted to python object
type(y_pred_val) # h2o.frame.H2OFrame

h2o.frame.H2OFrame

In [20]:
y_pred_val_df = pd.DataFrame(data =list(validation.index), columns = ["id"])
y_pred_val_df["price"] = y_pred_val.as_data_frame()
y_pred_val_df

,id,price
0,0,402.403412
1,1,2054.629622
2,2,9595.697945
3,3,555.585977
4,4,9529.741362
...,...,...
13444,13444,4754.795654
13445,13445,498.280992
13446,13446,14825.154211
13447,13447,10671.388985


In [ ]:
#del y_pred_df

Save the result:

In [21]:
y_pred_val_df.to_csv("../data/y_pred_14.csv", index=False)